In [1]:
import pandas as pd
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import pytz
from tqdm import tqdm
from scipy.spatial import KDTree

df=pd.read_csv("Thunderstorm_Wind.csv")
df_new=pd.read_csv("thunderstorm_var.csv")

# Prepare coordinates from both datasets
weather_coords = df_new[['event_latitude', 'event_longitude']].to_numpy()
tornado_coords = df[['BEGIN_LAT', 'BEGIN_LON']].dropna().to_numpy()

# Build KDTree for fast nearest-neighbor lookup
tree = KDTree(tornado_coords)

# Query the nearest tornado point for each weather point
distances, indices = tree.query(weather_coords, k=1)

# Extract matched indices
matched_tornado_data = df.iloc[indices.flatten()][['STATE', 'CZ_NAME']].reset_index(drop=True)

# Add county and state to weather dataset
df_new['state'] = matched_tornado_data['STATE']
df_new['county'] = matched_tornado_data['CZ_NAME']
df_new.to_csv("total_thunder_state.csv", index=False)


# Display updated dataframe
df_new[['event_latitude', 'event_longitude', 'state', 'county']].head(10)

,event_latitude,event_longitude,state,county
0,39.8800,-88.7300,INDIANA,PORTER
1,31.5000,-89.2800,VIRGINIA,CARROLL
2,31.8300,-89.0000,VIRGINIA,BOTETOURT
3,31.7100,-89.1400,VIRGINIA,CHARLOTTE
4,42.2500,-78.8000,NEW YORK,CATTARAUGUS
5,39.1132,-77.6949,NEW HAMPSHIRE,STRAFFORD
6,39.2601,-77.5801,ILLINOIS,PIATT
7,38.8716,-78.5255,MISSISSIPPI,JONES
8,39.7192,-76.6533,MISSISSIPPI,JASPER
9,39.3627,-77.3865,MISSISSIPPI,JONES


In [2]:
import pandas as pd
import os

# Load the CSV file
df = pd.read_csv('total_thunder_state.csv')

# Make sure 'time' column is datetime
df['time'] = pd.to_datetime(df['time'])

# Create output folder if it doesn't exist
output_folder = 'yearly_data'
os.makedirs(output_folder, exist_ok=True)

# Extract year from 'time' column
df['year'] = df['time'].dt.year

# Group by year and save each year's data
for year, group in df.groupby('year'):
    output_path = os.path.join(output_folder, f'data_{year}.csv')
    group.drop(columns='year').to_csv(output_path, index=False)

print("✅ Year-wise files created in 'yearly_data/' folder.")


✅ Year-wise files created in 'yearly_data/' folder.


In [3]:
import pandas as pd
import os
from datetime import timedelta
from tqdm import tqdm
import gc

# Settings
BUFFER_HOURS = 6
START_YEAR = 2014
END_YEAR = 2023
OUTPUT_FOLDER = "matched_output"
CHUNKSIZE = 100_000  # Adjust based on your system's memory

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Define dtypes to reduce memory usage
outage_dtypes = {
    'state': 'category',
    'county': 'category',
    'run_start_time': 'object'  # Will parse to datetime later
}
thunderstorm_dtypes = {
    'state': 'category',
    'county': 'category',
    'event_datetime': 'object'  # Will parse to datetime later
}

for year in range(START_YEAR, END_YEAR + 1):
    print(f"\n🔄 Processing year: {year}")

    # File paths
    outage_file = f"eaglei_data/eaglei_outages_{year}.csv"
    thunderstorm_file = f"yearly_data/data_{year}.csv"
    output_file = os.path.join(OUTPUT_FOLDER, f"thunderstorm_outage_{year}.csv")

    # Skip if either file is missing
    if not (os.path.exists(outage_file) and os.path.exists(thunderstorm_file)):
        print(f"⚠️ Missing files for {year}, skipping.")
        continue

    # Load thunderstorm data (assumed smaller, load entirely)
    try:
        thunderstorm_df = pd.read_csv(thunderstorm_file, dtype=thunderstorm_dtypes)
        thunderstorm_df['event_datetime'] = pd.to_datetime(
            thunderstorm_df['event_datetime'], format='%d/%m/%y %H:%M', errors='coerce'
        )
        if thunderstorm_df['event_datetime'].isna().sum() > 0:
            print(f"⚠️ {thunderstorm_df['event_datetime'].isna().sum()} invalid dates in thunderstorm data for {year}")
        thunderstorm_df['county'] = thunderstorm_df['county'].str.strip().str.lower()
        thunderstorm_df['state'] = thunderstorm_df['state'].str.strip().str.lower()
        thunderstorm_df['key'] = thunderstorm_df['state'] + "_" + thunderstorm_df['county']
        thunderstorm_df['event_id'] = thunderstorm_df.index  # Unique ID for each thunderstorm event
    except Exception as e:
        print(f"❌ Error in thunderstorm data for {year}: {e}")
        continue

    # Initialize results list for matched data
    matched_results = []

    # Process outage data in chunks
    try:
        # Only load required columns
        outage_chunks = pd.read_csv(
            outage_file,
            dtype=outage_dtypes,
            usecols=['state', 'county', 'run_start_time'],
            chunksize=CHUNKSIZE
        )
        for chunk in tqdm(outage_chunks, desc=f"Processing outages {year}"):
            # Parse datetime
            chunk['run_start_time'] = pd.to_datetime(chunk['run_start_time'], errors='coerce')
            if chunk['run_start_time'].isna().sum() > 0:
                print(f"⚠️ {chunk['run_start_time'].isna().sum()} invalid dates in outage chunk for {year}")

            # Normalize location
            chunk['county'] = chunk['county'].str.strip().str.lower()
            chunk['state'] = chunk['state'].str.strip().str.lower()
            chunk['key'] = chunk['state'] + "_" + chunk['county']

            # Merge with thunderstorm data
            merged_df = pd.merge(
                thunderstorm_df,
                chunk,
                on='key',
                suffixes=('_thunderstorm', '_outage')
            )

            # Filter by time difference
            time_diff = merged_df['run_start_time'] - merged_df['event_datetime']
            valid_time = (time_diff >= timedelta(0)) & (time_diff <= timedelta(hours=BUFFER_HOURS))
            filtered_df = merged_df[valid_time]

            if not filtered_df.empty:
                # Aggregate per thunderstorm event, only tracking event_id
                grouped = filtered_df[['event_id']].drop_duplicates().reset_index(drop=True)
                grouped['caused_power_outage'] = 1
                matched_results.append(grouped)

            # Free memory
            del chunk, merged_df, filtered_df
            gc.collect()

    except Exception as e:
        print(f"❌ Error in outage data for {year}: {e}")
        continue

    # Combine matched results
    if matched_results:
        final_matched = pd.concat(matched_results).groupby('event_id').agg({
            'caused_power_outage': 'max'
        }).reset_index()
    else:
        final_matched = pd.DataFrame(columns=['event_id', 'caused_power_outage'])

    # Merge back into thunderstorm_df
    thunderstorm_df = pd.merge(thunderstorm_df, final_matched, on='event_id', how='left')
    thunderstorm_df['caused_power_outage'] = thunderstorm_df['caused_power_outage'].fillna(0).astype(int)

    # Drop temp columns and save
    thunderstorm_df.drop(columns=['event_id', 'key'], inplace=True)
    thunderstorm_df.to_csv(output_file, index=False)
    print(f"✅ Saved: {output_file} | Matches: {thunderstorm_df['caused_power_outage'].sum()}")

    # Cleanup
    del thunderstorm_df, final_matched, matched_results
    gc.collect()

python(16374) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



🔄 Processing year: 2014


Processing outages 2014: 0it [00:00, ?it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['run_start_time'] = pd.to_datetime(chunk['run_start_time'], errors='coerce')
Processing outages 2014: 1it [00:00,  3.17it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['run_start_time'] = pd.to_datetime(chunk['run_start_time'], errors='coerce')
Processing outages 2014: 2it [00:00,  3.69it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed individuall

✅ Saved: matched_output/thunderstorm_outage_2014.csv | Matches: 498

🔄 Processing year: 2015


Processing outages 2015: 21it [00:04,  5.08it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['run_start_time'] = pd.to_datetime(chunk['run_start_time'], errors='coerce')
Processing outages 2015: 40it [00:07,  5.33it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['run_start_time'] = pd.to_datetime(chunk['run_start_time'], errors='coerce')
Processing outages 2015: 41it [00:08,  5.30it/s]/var/folders/46/svrn09zn5wq75vy8775dkw440000gn/T/ipykernel_9628/1668799726.py:72: UserWarning: Could not infer format, so each element will be parsed indi

...

✅ Saved: matched_output/thunderstorm_outage_2021.csv | Matches: 11318

🔄 Processing year: 2022


Processing outages 2022: 258it [00:51,  5.00it/s]


✅ Saved: matched_output/thunderstorm_outage_2022.csv | Matches: 12493

🔄 Processing year: 2023


Processing outages 2023: 262it [01:00,  4.34it/s]


✅ Saved: matched_output/thunderstorm_outage_2023.csv | Matches: 15553


In [4]:
import pandas as pd
import glob
import os

# Path to the folder containing the CSV files
folder_path = 'matched_output'

# Create a list of file paths for the years 2014 to 2023
csv_files = [os.path.join(folder_path, f'thunderstorm_outage_{year}.csv') for year in range(2014, 2024)]

# Read and concatenate all the CSV files
combined_df = pd.concat([pd.read_csv(file) for file in csv_files if os.path.exists(file)], ignore_index=True)

# Optional: Save the combined DataFrame to a new CSV
combined_df.to_csv('thunderstorm_outage_combined_2014_2023.csv', index=False)

print(f"Combined {len(csv_files)} files into one DataFrame with {len(combined_df)} rows.")


Combined 10 files into one DataFrame with 162229 rows.
